# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [10]:
# Your code here
# cur.execute('''SELECT DISTINCT city FROM offices''').fetchall()

cur.execute('''SELECT firstName, lastName, city 
                FROM employees e JOIN offices USING(officeCode) 
                WHERE city = "Boston"''')
pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

,firstName,lastName,city
0,Julie,Firrelli,Boston
1,Steve,Patterson,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [14]:
# Your code here
cur.execute('''SELECT COUNT(employeeNumber), city
                FROM employees JOIN offices USING(officeCode)
                GROUP BY city
                HAVING count(employeeNumber) = 0''')
pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

,COUNT(employeeNumber),city


## Write 3 Questions of your own and answer them

In [32]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
# cur.execute('''SEELECT productlines.htmlDescription, employees.firstName, employees.lastName
#                 FROM ''')

cur.execute('''SELECT productlines.htmlDescription, employees.firstName, employees.lastName, orders.orderNumber
                FROM customers 
                    JOIN employees ON employees.employeeNumber = customers.salesRepEmployeeNumber
                    JOIN orders USING(customerNumber)
                    JOIN orderdetails USING(orderNumber)
                    JOIN products USING(productCode)
                    JOIN productlines USING(productLine)
                    WHERE htmlDescription IS NOT ""''')


pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

,htmlDescription,firstName,lastName,orderNumber


In [36]:
## are there any thmlDescriptions?
cur.execute('''SELECT productlines.htmlDescription FROM productlines''')
cur.fetchall()

#nope

[('',), ('',), ('',), ('',), ('',), ('',), ('',)]

In [41]:
cur.description

(('firstName', None, None, None, None, None, None),
 ('lastName', None, None, None, None, None, None),
 ('MAX(amount)', None, None, None, None, None, None))

In [43]:
# Your code here
# What is the highest customer payment for each employee
cur.execute('''SELECT firstName, lastName, MAX(amount)
                FROM employees
                JOIN customers ON employeeNumber = salesRepEmployeeNumber
                JOIN payments USING(customerNumber)
                GROUP BY employeeNumber
                ORDER BY MAX(amount) DESC''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])


,firstName,lastName,MAX(amount)
0,Gerard,Hernandez,120166.58
1,Leslie,Jennings,111654.40
2,Mami,Nishi,105743.00
3,Barry,Jones,85024.46
4,Andy,Fixter,82261.22
5,Leslie,Thompson,80375.24
6,Peter,Marsh,75020.13
7,Steve,Patterson,63357.13
8,Martin,Gerard,61402.00
9,Julie,Firrelli,59265.14


In [62]:
# Your code here
# Which product does each employee sell the most of
cur.execute('''SELECT employeeNumber, firstName, lastName, MAX(c), productCode, productName, productLine
                FROM
                    (SELECT employeeNumber, firstName, lastName, COUNT(productCode) as c, productCode, productName, productLine
                    FROM employees
                    JOIN customers on employeeNumber = salesRepEmployeeNumber
                    JOIN orders USING(customerNumber)
                    JOIN orderdetails USING(orderNumber)
                    JOIN products USING(productCode)
                    GROUP BY employeeNumber, productCode)
                GROUP BY employeeNumber''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

# this one seems more complicated then necessary but I couldn't figure out how to take the max of count without
# nesting queries

,employeeNumber,firstName,lastName,MAX(c),productCode,productName,productLine
0,1165,Leslie,Jennings,8,S18_3320,1917 Maxwell Touring Car,Vintage Cars
1,1166,Leslie,Thompson,3,S18_1589,1965 Aston Martin DB5,Classic Cars
2,1188,Julie,Firrelli,4,S24_2841,1900s Vintage Bi-Plane,Planes
3,1216,Steve,Patterson,4,S18_3232,1992 Ferrari 360 Spider red,Classic Cars
4,1286,Foon Yue,Tseng,5,S18_4027,1970 Triumph Spitfire,Classic Cars
5,1323,George,Vanauf,7,S32_1374,1997 BMW F650 ST,Motorcycles
6,1337,Loui,Bondur,5,S32_2206,1982 Ducati 996 R,Motorcycles
7,1370,Gerard,Hernandez,11,S18_3232,1992 Ferrari 360 Spider red,Classic Cars
8,1401,Pamela,Castillo,6,S700_1938,The Mayflower,Ships
9,1501,Larry,Bott,6,S18_3232,1992 Ferrari 360 Spider red,Classic Cars


In [64]:
# Your code here
# How many customers does each employee have in each city?
cur.execute('''SELECT firstName, lastName, COUNT(customerNumber) as numCustomers, city, state, country
                FROM employees
                JOIN customers on employeeNumber = salesRepEmployeeNumber
                GROUP BY country, city, state''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])


,firstName,lastName,numCustomers,city,state,country
0,Andy,Fixter,1,Chatswood,NSW,Australia
1,Andy,Fixter,1,Glen Waverly,Victoria,Australia
2,Andy,Fixter,1,Melbourne,Victoria,Australia
3,Andy,Fixter,1,North Sydney,NSW,Australia
4,Andy,Fixter,1,South Brisbane,Queensland,Australia
...,...,...,...,...,...,...
75,Leslie,Thompson,1,San Diego,CA,USA
76,Leslie,Jennings,2,San Francisco,CA,USA
77,Leslie,Jennings,1,San Jose,CA,USA
78,Leslie,Jennings,1,San Rafael,CA,USA


## Level Up: Display the names of every individual product that each employee has sold

In [66]:
# Your code here
cur.execute('''SELECT DISTINCT firstName, lastName, productName
                FROM products
                JOIN orderDetails USING(productCode)
                JOIN orders USING(orderNumber)
                JOIN customers USING (customerNumber)
                JOIN employees on employeeNumber = salesRepEmployeeNumber
                ORDER BY lastName, firstName''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

,firstName,lastName,productName
0,Loui,Bondur,1969 Harley Davidson Ultimate Chopper
1,Loui,Bondur,1952 Alpine Renault 1300
2,Loui,Bondur,1996 Moto Guzzi 1100i
3,Loui,Bondur,2003 Harley-Davidson Eagle Drag Bike
4,Loui,Bondur,1972 Alfa Romeo GTA
...,...,...,...
1363,George,Vanauf,The Titanic
1364,George,Vanauf,The Queen Mary
1365,George,Vanauf,American Airlines: MD-11S
1366,George,Vanauf,Boeing X-32A JSF


## Level Up: Display the Number of Products each employee has sold

In [72]:
# Your code here
# Assuming "Number of Products" means "number of different products" and not total count of items sold.

cur.execute('''SELECT employeeNumber, firstName, lastName, productCode, COUNT(productCode) as numSold
                FROM products
                JOIN orderDetails USING(productCode)
                JOIN orders USING(orderNumber)
                JOIN customers USING (customerNumber)
                JOIN employees on employeeNumber = salesRepEmployeeNumber
                GROUP BY employeeNumber, productCode
                ''')

pd.DataFrame(cur.fetchall(), columns = [x[0] for x in cur.description])

,employeeNumber,firstName,lastName,productCode,numSold
0,1165,Leslie,Jennings,S10_1678,3
1,1165,Leslie,Jennings,S10_1949,5
2,1165,Leslie,Jennings,S10_2016,3
3,1165,Leslie,Jennings,S10_4698,4
4,1165,Leslie,Jennings,S10_4757,2
...,...,...,...,...,...
1363,1702,Martin,Gerard,S700_2834,1
1364,1702,Martin,Gerard,S700_3505,3
1365,1702,Martin,Gerard,S700_3962,2
1366,1702,Martin,Gerard,S72_1253,1


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!